# pybulletとは
pybulletは、物理シミュレーションエンジンの一つであり、ロボットの制御や物理シミュレーションを行うためのライブラリです。

ロボットのシミュレーションや制御にはROS（Robot Operating System）が使われることが多いですが、ROSは環境構築が複雑です。一方で、pybulletはpythonのライブラリなので、pythonが使える環境であれば簡単に扱うことができます。

こちらでは、基本的な機能のみを紹介します。  
pybulletで使用可能な関数がまとめられたマニュアルについては[こちら](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf)を参照してください。

# 必要なライブラリのインポート

pybulletを使用する際は、`pybullet`をインポートする必要があります。   
また、pybulletに使用するファイルを読み込むために`import pybullet_data`もインポートします。

In [1]:
import pybullet
import pybullet_data

pybullet build time: Nov 28 2023 23:45:17


# pybulletの起動
pybulletを使用する際は、`pybullet.connect`を使用して、物理シミュレーションを行うためのサーバーを起動します。

サーバの種類には以下の種類が存在します
- `pybullet.GUI`
- `pybullet.DIRECT`
- `pybullet.SHARED_MEMORY`
- `pybullet.UDP`
- `pybullet.TCP`

基本的には、CUIで使用する場合は`pybullet.DIRECT`、GUIで使用する場合は`pybullet.GUI`を使用します。

<br>

今回はGUIを使用するため、`pybullet.GUI`を指定してサーバーを起動します。
以下セルを実行すると、pybulletのGUIが起動されます。  
（google colabなどではGUIが表示されないため、`pybullet.DIRECT`を指定してサーバーを起動する必要があります。）

In [2]:
physicsClient = pybullet.connect(pybullet.GUI) 

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Microsoft Corporation
GL_RENDERER=D3D12 (AMD Radeon Graphics)
GL_VERSION=4.2 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.20
pthread_getconcurrency()=0
Version = 4.2 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Microsoft Corporation
Renderer = D3D12 (AMD Radeon Graphics)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


ven = Microsoft Corporation
ven = Microsoft Corporation


# シミュレーションの初期設定
シミュレーションを開始するにあたって、初期設定を行います。
- シミュレーション空間のリセット
- pybulletに必要なデータへのパスの追加
- 重力の設定
- 1stepあたりに経過する時間（秒単位）

In [10]:
pybullet.resetSimulation() # シミュレーション空間をリセット
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # pybulletに必要なデータへのパスを追加
pybullet.setGravity(0.0, 0.0, -9.8) # 地球上における重力に設定
timeStep = 1./240.
pybullet.setTimeStep(timeStep) # 1stepあたりに経過する時間の設定

# 床の読み込み
pybulletでは、標準で幾つかのモデルが用意されています。ここでは、その中で床が定義されているファイルである `plane.urdf`を読み込みます。  

---

urdfファイルは「ロボットモデルについて定義されるxml形式のファイル」になり、ロボットのリンクやジョイントについて定義できます。  
基本的には、ロボットに関して定義するものになりますが、`plane.urdf`では、ロボットではなく床のモデルが定義されています。

---

In [11]:
#床の読み込み
planeId = pybullet.loadURDF("plane.urdf")

# 物体の読み込み
`createCollisionShape` `createVisualShape` `createMultiBody`関数を使用することで、任意のサイズのオブジェクト（ボックスや球など）

In [12]:
# ボックスの読み込み
## ボックスの重さ、サイズ、位置·姿勢を決める
mass = 5 # kg
box_size = [0.3, 0.3, 0.3]
position = [2, 0, 0.3]
orientation = [1, 0, 0, 0] # 四元数
box_cid = pybullet.createCollisionShape(pybullet.GEOM_BOX, halfExtents=box_size, physicsClientId=physicsClient)
box_vid = pybullet.createVisualShape(pybullet.GEOM_BOX, halfExtents=box_size, physicsClientId=physicsClient, rgbaColor=[1,0,0,1]) # 赤・半透明
box_bid = pybullet.createMultiBody(mass, box_cid, box_vid, position, orientation, physicsClientId=physicsClient)

# ロボットの読み込み
自身で定義したロボットのurdfファイルを読み込みます。
`loadURDF`関数の引数に「urdfファイルまでのパス」を指定することでロボットを生成することができます

In [13]:
# ロボットの読み込み
carStartPos = [0, 0, 0.1]  # 初期位置(x,y,z)を設定
carStartOrientation = pybullet.getQuaternionFromEuler([0,0,0])  # 初期姿勢(roll, pitch, yaw)を設定
# urdfファイルのmeshはテクスチャが反映されないっぽいので各linkにrgbaタグで色を付けている
carId = pybullet.loadURDF("../urdf/simple_two_wheel_car.urdf",carStartPos, carStartOrientation)

# カメラ位置の設定
`resetDebugVisualizerCamera`関数で、GUIモードの際のカメラを設定できます。

In [14]:
# GUIモードの際のカメラの位置などを設定
cameraDistance = 4.0
cameraYaw = 0.0 # deg
cameraPitch = -20 # deg
cameraTargetPosition = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(cameraDistance, cameraYaw, cameraPitch, cameraTargetPosition)

# デバック用の文字表示
`addUserDebugText`関数を使用することで、シミュレーション空間上の任意の位置にテキストを表示させることができます。  

他にも、`addUserDebugLine`では、シミュレーション空間上の任意の2点間を結ぶを結ぶ線を描画することができ、移動ロボットの移動経路やロボットアームの手先の軌跡などを描画する際に便利です。

このように、pybulletではシミュレーション結果を可視化するのに便利な機能が搭載されています。

In [15]:
# 画面上に文字を表示
textPosition = [0.0, 0.0, 2.0]
lifeTime = 10.0 # 表示期間（秒）
pybullet.addUserDebugText(f"test text", textPosition, textSize=2, lifeTime=lifeTime)

0

# シミュレーションの実行
`stepSimulation`関数を使用すると、`setTimeStep`で設定した時間分シミュレーション空間上で時間が経過します。  
ここでは、移動ロボットの両輪に速度指令を与えて200時刻分シミュレーションを実行しています。


In [22]:
import time
rightWheelJointIdx = 0
leftWheelJointIdx = 1
for i in range(200):
    pybullet.setJointMotorControl2(carId, leftWheelJointIdx, pybullet.VELOCITY_CONTROL, targetVelocity=10)
    pybullet.setJointMotorControl2(carId, rightWheelJointIdx, pybullet.VELOCITY_CONTROL, targetVelocity=10)
    pybullet.stepSimulation()
    time.sleep(timeStep)